Gathering all input variables, design parameters, and actual constants in one place.

In [3]:
global OF, rho_ipa # GLOBAL IS EVIL

# Physics
g_n = 9.80665  # kg.m/s^2     Standard gravity
R_const = 8314.46261815324 # gas constant, J/ (K * kmol)
def spec_gas(M): return R_const / M

# Chemistry, we are using LOX and IPA/H20
rho_lox = 1141.0   # kg/m^3  Density of LOX
rho_ipa = 849.28   # kg/m^3  Density of 64.8% IPA / 35.2% H20

# Tank Materials
# https://www.aircraftspruce.com/catalog/cmpages/anh4120honeycomb01-01574.php?clickkey=5444217
# i've calculated ~395 kg/m^3 for uniform airframe density based on our density/thickness estimates below.
# measurement of LV3.1 says module is around 130 kg/m^3
rho_nomex = 48.06 # kg/m^3, density of 3/16th in overexpanded nomex honeycomb
rho_cryo  = 160 # kg/m^3, density of cryogel-z insulation
rho_fiber = 1850 # kg/m^3, density of fiberglass

Al    = { 'rho': 2800.0,    # kg/m^3       Density
          'Sy':    0.270e9} # Pa           Yield strength
CF    = { 'rho': 1550.0,    # kg/m^3       Density
          'Sy':    0.450e9} # Pa           Yield strength

# Launch constants
launch_site_alt = 1401 # m, altitude of launch site above sea level
launch_tower = 60. # launch rail height in ft
min_throttle = 0.65 # the internet says between 60 - 70% is doable without ruining our lives

In [4]:
# combustion gas properties ke, Re, T_ch, determined from CEArun
# with chamber pressure=350 psi, fuel temp=419.15 K, 
#      lox temp=90 K, OF=1.3 for fuel = 64.8% IPA (2propanol) / 35.2% H20
OF   = 1.3        # O/F ratio, this is somewhat arbitrary but CEA says its good. 
p_ch = 350 # chamber pressure, PSI
T_ch = 3097.82 # chamber temperature, K
ke   = 1.1251 # specific heat ratio, propellant (aka gammas)
M    = 23.196 # molar mass
Re   = spec_gas(M) # specific gas constant, propellant

# willy wonka's fudge factors
ullage = 1.1          # percentage of length added to a tank to account for not filling
factor_of_safety = 2  # factor of safety
mass_fudger = 2       # fudge factor for design mass, includes contribution of tank structural lugs,
                      # feed system, stress concentrations, welds, slosh baffles etc.
airframe_fudger = 1.1 # arbitrary number to multiply all airframe+subsystem masses by
loss_factor = 1.      # if < 1, then assume exhaust velocity is less than ideal (percentage)

In [ ]:
# airframe parameters
airframe_offset      = 0.007    # m, empirical from lv3, gap between airframe inner diameter and innards
nomex_thickness      = 0.00635  # m, from lv3.1 (0.25 in)
cf_thickness         = 0.001905    # m, from lv3.1 (0.075 in)
airframe_thickness   = nomex_thickness + cf_thickness # m, thiccness of lv3.1
insulation_layers    = 2 # around LOX tank, likely need from 1-3
insulation_thickness = 0.005 # m
dist_after_f = 0.0      # m, we could place fuel tank before avionics and N2, but don't because bad idea
dist_after_o = 0.0      # m, oxygen tank still in front of feedsys, ems, engine (duh)

# upper subsystem module dimensions
nose_l = 1.50        # m
ers_l  = 6 * 0.0254  # m (converted from in)
rcs_l  = 6 * 0.0254  # m (converted from in)
av_l   = 18 * 0.0254 # m (converted from in)
n2_l   = 18 * 0.0254 # m (converted from in)
gaps   = 0.050      # m, space between subsystems

# internal subsystem masses
m_nose_frame  = 3               # nosecone outer frame weight [kg] (est from openrocket)
m_ballast     = 8               # ballast weight [kg]
m_recovery    = 4               # Recovery system mass estimate [kg] (lv3.1 measured at 3.5 kg)
m_rcs         = 4               # guess at mass, totally made up  [kg]
m_n2          = 4               # mass of n2 tank [kg] (measured)
m_avionics    = 3.3             # Avionics mass  [kg]
m_fins        = 8.77            # total fin mass [kg], estimate from openrocket
m_ringsclamps = (.466 + 1) * 7  # weights of rings and clamps [kg]

# Engine system dimensions. 'm_' = mass, 'l_' = length
m_feed   =  10.0           # kg
l_feed   =  0.4572         # m, this is 18"
m_ems    =  1              # kg
l_ems    =  0.1016         # m, this is 4" 
m_engine =  3.0            # kg
l_engine =  0.300          # m

In [ ]:
# SIMULATION AND OPTIMIZATION PARAMETERS
delta      = 10**(-8)  # a guess at a good margin for design "convergence"
mu_0       = 0.0025    # barrier parameter, this value lets altitudes approach lower bound pretty quickly
rho_0      = 2.5       # penalty parameter, i'm still playing with this value.
time_step  = 0.25      # change time-step for trajectory simulation
iterations = 25        # number of escalating iteration sequences

# INITIAL DESIGN GUESS
# be sure that you start with a feasible design, otherwise the problem will be ill-conditioned
m_prop = 135. # propellant mass (kg)
mdot_0 = 3.4  # Propellant mass flow rate (kg/s)
dia    = 12.  # Rocket diameter (in)
p_e    = 55.  # Exit Pressure (kPa)
lower  = 100. # lower and upper bounds of drag force (N) for throttling
upper  = 1000.

# OPTIMIZATION CONSTRAINTS
cons_mass    = 250.                      # GLOW constraint, kg, somewhat arbitrary
cons_ls      = 22.                       # min launch speed from 60' tower constraint, m/s
cons_TWR     = 2.                        # TWR constraint
cons_S_crit  = 0.35                      # Critical pressure ratio constraint
cons_accel   = 15.                       # Max acceleration constraint, g's
cons_LD      = 30.                       # L/D ratio constraint, slightly arbitrary
cons_alt     = 120000. + launch_site_alt # Min altitude constraint, m (adjusted to overshoot)
cons_thrust  = 10.                        # max ground-level thrust, kN
cons_ceiling = 150000. + launch_site_alt # base-11 maximum apogee requirement, km
can_throttle = False